In [1]:
import requests
import shutil
import json
from pyproj import CRS, Transformer
import math
import numpy as np
import csv
import pandas as pd
from scipy import spatial

In [2]:
class RoadSegment():
    def __init__(
            self, 
            route_id,
            geometry):
        self.route_id = route_id
        self.num_pts = len(geometry)
        
        self.pts = [(coords[1], coords[0]) for coords in geometry]

        #Project the points from road data onto the same projection as aerial imagery
        crs_4326 = CRS('epsg:4326')
        crs_proj = CRS('epsg:26985')
        transformer = Transformer.from_crs(crs_4326, crs_proj)
        pts_proj = transformer.itransform(self.pts)
        self.pts_proj = [pt for pt in pts_proj]
        
        #Calculate the distance between each section in the segment
        self.sub_distances = []
        for i, coords in enumerate(self.pts_proj):
            if i == len(self.pts_proj) - 1:
                break
            x1, y1 = coords
            x2, y2 = self.pts_proj[i+1]
            distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            self.sub_distances.append(distance)
        self.total_distance = sum(self.sub_distances)
    

In [3]:
with open('./data/roadways/Roadway_Block.geojson') as f:
    dc_roadway_data = json.load(f)

In [4]:
dc_road_segments = []
for segment in dc_roadway_data['features']:
    if segment['geometry']['type'] == 'MultiLineString':
        for LineString in segment['geometry']['coordinates']:
            segment_obj = RoadSegment(
                segment['properties']['ROUTEID'],
                LineString
            )
            dc_road_segments.append(segment_obj)
    else:
        segment_obj = RoadSegment(
                    segment['properties']['ROUTEID'],
                    segment['geometry']['coordinates']
        )
        dc_road_segments.append(segment_obj)

In [5]:
#find a point some distance between p1 and p2
def interp_pts(p1, p2, dist):
    x1, y1 = p1
    x2, y2 = p2
    total_dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    x3 = x1 + (dist / total_dist) * (x2 - x1)
    y3 = y1 + (dist / total_dist) * (y2 - y1)

    return x3, y3

In [6]:
def collect_img_origins(segment, img_dim, overlap):
    img_origins = [segment.pts_proj[0], segment.pts_proj[-1]]
    #automatically want images of both ends of segment subtract image dim to account for this
    #divide by the dimension of an image minus overlap to determine number of images to take of segment
    num_imgs = math.ceil((segment.total_distance - img_dim + (overlap * 2)) / (img_dim - (overlap * 2)))
    if num_imgs == 0:
        return img_origins

    #Since we're rounding up, adjust the increment to evenly space out the images
    increment = segment.total_distance / (num_imgs + 1)
    #Find the distance from the start that each image should be taken at
    img_distances = [(i+1) * increment for i in range(0, num_imgs)]    
    #Find the section that each image should be centered around
    sections = []
    section_idx = 0
    section_distance = segment.sub_distances[section_idx]
    for distance in img_distances:
        while distance > section_distance:
            section_idx += 1
            section_distance += segment.sub_distances[section_idx]
        sections.append((distance, section_idx))
    dist_accumulator = 0
    accumulated_dists = [0]
    for dist in segment.sub_distances:
        dist_accumulator += dist
        accumulated_dists.append(dist_accumulator)
    #Find the center point that each image should be taken around
    for distance, section_idx in sections:
        p1 = segment.pts_proj[section_idx]
        p2 = segment.pts_proj[section_idx + 1]
        dist = distance - accumulated_dists[section_idx]
        img_pt = interp_pts(p1, p2, dist)
        img_origins.append(img_pt)

    return img_origins

In [40]:
BBOX_DIM = 60
OVERLAP = 3

all_img_coords = []
for segment in dc_road_segments:
    seg_img_coords = collect_img_origins(segment, BBOX_DIM, OVERLAP)
    all_img_coords.extend(seg_img_coords)
all_img_coords = np.asarray(all_img_coords)

In [38]:
def find_clusters_kd(img_coords, min_dist):
    kd_tree = spatial.KDTree(img_coords)
    clusters_raw = kd_tree.query_ball_point(img_coords, min_dist)
    #Remove duplicates
    clusters_set = {tuple(cluster) for cluster in clusters_raw}
    return clusters_set

In [10]:
def find_clusters(img_coords, min_dist):
    #Find all clusters
    #Ensure clusters are not duplicates in different orders
    clusters = set()
    for i, coord1 in enumerate(img_coords):
        x1, y1 = coord1
        cluster = [i]
        for j, coord2 in enumerate(img_coords):
            if i == j:
                continue
            x2, y2 = coord2
            distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            if distance < min_dist:
                cluster.append(j)
                
        #Ensure some clusters are not subsets of other clusters
        if len(cluster) > 1:
            cluster = tuple(sorted(cluster))
            cluster_set = set(cluster)
            subset = False
            for prev_cluster in clusters:
                if cluster_set.issubset(prev_cluster):
                    subset = True
                    break
            if not subset:
                clusters.add(cluster)
    
#     clusters_nosubsets = []
#     for i, cluster1 in enumerate(clusters):
#         subset = False
#         for j, cluster2 in enumerate(clusters):
#             if i == j:
#                 continue
#             if cluster1.issubset(cluster2):
#                 subset = True
#                 break

#         if not subset:
#             clusters_nosubsets.append(cluster1)

    return clusters

In [9]:
def merge_clusters(clusters, img_coords):
    new_coords = []
    clustered_pts = set()
    for cluster in clusters:
        mean_x = 0
        mean_y = 0
        for pt_idx in cluster:
            x, y = img_coords[pt_idx]
            mean_x += x
            mean_y += y
            clustered_pts.add(pt_idx)
        mean_x /= len(cluster)
        mean_y /= len(cluster)        
        new_coords.append((mean_x, mean_y))
    
    coords_idxs = set(np.arange(0, len(img_coords), 1))
    non_clusterd_pts = coords_idxs.difference(clustered_pts)
    for pt_idx in non_clusterd_pts:
        new_coords.append(img_coords[pt_idx])
    return np.asarray(new_coords)

In [10]:
def project_to_latlng(pt):
    crs_4326 = CRS('epsg:4326')
    crs_proj = CRS('epsg:26985')
    transformer = Transformer.from_crs(crs_proj, crs_4326)
    pt_proj = transformer.transform(pt[0], pt[1])
    
    return pt_proj

In [42]:
coord_clusters = find_clusters_kd(all_img_coords, 20)
merged_coords = merge_clusters(coord_clusters, all_img_coords)
len(merged_coords)

36192

In [43]:
coord_clusters2 = find_clusters_kd(merged_coords, 20)
merged_coords2 = merge_clusters(coord_clusters2, merged_coords)
len(merged_coords2)

36076

In [14]:
#latlng_coords = [project_to_latlng(coord) for coord in merged_coords]

In [44]:
coord_clusters1 = list(coord_clusters)
coord_clusters2 = list(coord_clusters)

for i, m in enumerate(coord_clusters1):
    for j, n in enumerate(coord_clusters1):
        if set(m).issubset(set(n)) and i != j:
            coord_clusters2.remove(m)
            break

In [13]:
def convert_to_bbox(img_coord, dim):
    r = dim / 2
    xmin = img_coord[0] - r
    xmax = img_coord[0] + r
    ymin = img_coord[1] - r
    ymax = img_coord[1] + r
    
    return (xmin, ymin, xmax, ymax)

In [44]:
image_bboxes = [(convert_to_bbox(coord, BBOX_DIM), tuple(coord)) for coord in merged_coords]
region_bbox = (399697, 135518, 401430, 136935)
region_data = [coord for coord in image_bboxes if coord[0][0] > region_bbox[0] and coord[0][1] > region_bbox[1]
                    and coord[0][2] < region_bbox[2] and coord[0][3] < region_bbox[3]]

In [45]:
bboxes_in_region = [coord[0] for coord in region_data]
center_coords_region = [coord[1] for coord in region_data]
latlng_in_region = [project_to_latlng(coord) for coord in center_coords_region]

In [46]:
df_region = pd.DataFrame(bboxes_in_region, columns=['xmin', 'ymin', 'xmax', 'ymax'])
image_filenames = ['image_' + str(i) + '.png' for i in range(len(bboxes_in_region))]
df_region['filename'] = image_filenames
df_region['lat'] = [coord[0] for coord in latlng_in_region]
df_region['lng'] = [coord[1] for coord in latlng_in_region]
df_region['center_x'] = [coord[0] for coord in center_coords_region]
df_region['center_y'] = [coord[1] for coord in center_coords_region]
df_region['overlap'] = OVERLAP

In [48]:
df_region.to_csv('399697_135518_401430_136935.csv', index=False)

In [58]:
def cluster_high_dim_images(img_coords, img_dim, high_dim=4100):
    r = high_dim / 2 - (img_dim / 2)
    kd_tree = spatial.KDTree(img_coords)
    clusters_raw = kd_tree.query_ball_point(img_coords, r)
    clusters_set = {tuple(cluster) for cluster in clusters_raw}
    return clusters_set

In [60]:
def removeSublists(l):
    for m in l:
        for n in l:
            if set(m).issubset(set(n)) and m != n:
                l2.remove(m)
                break

In [61]:
high_dim_img_clusters = cluster_high_dim_images(merged_coords, 100)

In [ ]:
high_dim_img_clusters_lst1 = [cluster for cluster in high_dim_img_clusters]
high_dim_img_clusters_lst2 = [cluster for cluster in high_dim_img_clusters]

for m in high_dim_img_clusters_lst1:
        for n in high_dim_img_clusters_lst1:
            if set(m).issubset(set(n)) and m != n:
                high_dim_img_clusters_lst2.remove(m)
                break

In [ ]:
merged = removeSublist(high_dim_img_clusters)